In [1]:
import numpy as np
from functools import partial
from pathlib import Path 
import shutil

import datasets
from torchvision import transforms

/Users/alexgrig/miniconda/envs/fedml/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%ls -al /Users/alexgrig/data/fedml/mnist/train/

total 1914752
drwxrwxr-x@  3 alexgrig  staff         96 Nov 28  2018 ./
drwxr-xr-x  10 alexgrig  staff        320 Oct 12 12:21 ../
-rw-r--r--@  1 alexgrig  staff  971880155 Nov 28  2018 all_data_0_niid_0_keep_10_train_9.json


In [3]:
"""
Existing Functions:

list_datasets
load_dataset_builder # Explore dataset
load_dataset # Actually load it
get_dataset_split_names # Explore dataset. Get split names
get_dataset_config_names # Explore dataset. Get config (subdatasets) names

concatenate_datasets
remove_columns
"""

'\nExisting Functions:\n\nlist_datasets\nload_dataset_builder # Explore dataset\nload_dataset # Actually load it\nget_dataset_split_names # Explore dataset. Get split names\nget_dataset_config_names # Explore dataset. Get config (subdatasets) names\n\nconcatenate_datasets\nremove_columns\n'

In [4]:
[dd for dd in datasets.list_datasets() if 'mnist' in dd]

['fashion_mnist',
 'mnist',
 'Fraser/mnist-text-default',
 'Fraser/mnist-text-no-spaces',
 'Fraser/mnist-text-small',
 'cgarciae/point-cloud-mnist',
 'lewtun/mnist-preds',
 'msollami-sf/processed_mnist',
 'mweiss/fashion_mnist_corrupted',
 'flexthink/audiomnist',
 'autoevaluate/mnist-sample',
 'chrisjay/mnist-adversarial-dataset',
 'arize-ai/fashion_mnist_label_drift',
 'chrisjay/test-mnist',
 'arize-ai/fashion_mnist_quality_drift',
 'chrisjay/test-mnist-2',
 'chrisjay/test-mnist-3',
 'chrisjay/test-mnist-4',
 'chrisjay/test-mnist-5',
 'Depshad/ood_mnist_c']

In [5]:
mnist = datasets.load_dataset('mnist')
ds_test = datasets.load_dataset('mnist', split='test[0:5]')

Found cached dataset mnist (/Users/alexgrig/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 451.78it/s]
Found cached dataset mnist (/Users/alexgrig/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)


In [6]:
mnist, ds_test

(DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 60000
     })
     test: Dataset({
         features: ['image', 'label'],
         num_rows: 10000
     })
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 5
 }))

In [7]:
ds_test.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], id=None)}

## Filter on labels and substitute labels (make the problem binary):

In [12]:
use_labels = [3,8]
new_labels = [0,1]

In [13]:
mnist = mnist.filter(lambda sample: sample['label'] in use_labels, load_from_cache_file=False)

 50%|███████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                               | 1/2 [00:00<00:00,  4.52ba/s]


In [14]:
mnist

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11982
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1984
    })
})

In [15]:
# Check filtering
np.unique( mnist['test']['label'] ), np.unique( mnist['train']['label'] )

(array([3, 8]), array([3, 8]))

## Map lables to new labels:

In [16]:
def substitute(sample):
    
    sample['label'] = (new_labels[0] if (sample['label']==use_labels[0]) else new_labels[1])
    return sample

mnist = mnist.map(substitute)

Parameter 'function'=<function substitute at 0x7f8ab4380a70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1984/1984 [00:00<00:00, 12515.79ex/s]


In [17]:
# Check substitution
np.unique( mnist['test']['label'] ), np.unique( mnist['train']['label'] )

(array([0, 1]), array([0, 1]))

## Split train part into val and train parts:

### Make validation part:

In [18]:
make_validation = False

test_size = 0.15
stratify_by_column='label'
shuffle=True
seed=47

In [19]:
def make_train_val_split(ds, test_size=0.15, stratify_by_column='label', shuffle=True, seed=47):
    d_dict = ds.train_test_split(test_size=test_size, stratify_by_column=stratify_by_column, shuffle=shuffle, seed=seed)
    
    train_train = d_dict['train']
    train_train.split._name = 'train_train'
    
    train_val = d_dict['test']
    train_val.split._name = 'train_val'
    
    return train_train, train_val

# divide onto train and val
if make_validation:
    train_train, train_val = make_train_val_split(mnist['train'], test_size=test_size, stratify_by_column=stratify_by_column, shuffle=shuffle, seed=seed)
    mnist = datasets.DatasetDict({'train_train': train_train, 'train_val': train_val, 'test': mnist['test']})
mnist

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11982
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1984
    })
})

In [20]:
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()

def split_image(image, parts=2):
    """
    Splits the image onto several parts. Has not been tested when the number of parts larger than 5.
    So, there may be border effects.
    """
    
    split_dim = 1
    split_dim_size = image.shape[split_dim] 
    
    part_size = int( np.ceil(split_dim_size / parts) )
    split_points = np.arange(0, split_dim_size, part_size)
    #print(split_points)
                             
    image_parts = []
    start_points = list(split_points[0:])
    end_points = list(split_points[1:]) + [split_dim_size+1,]                          
    for st,en in zip(start_points, end_points):
        part = image[st:en,:] if (split_dim == 0) else image[:,st:en]
        image_parts.append(part)
        
    return image_parts


def split_vertically(sample, parts=3, split_feature='image', part_prefix='image_part'):
    
    image = sample[split_feature] # make sure it is decoded from the storing format
    #import pdb; pdb.set_trace()
    timage = to_tensor(image)[0,:,:]
    
    image_parts = { f'{part_prefix}_{ii}':to_image(img) for ii,img in enumerate(split_image(timage, parts=parts)) }
    
    return image_parts

## Test the splitting over a features into parts:

### Set splitting parameters:

In [21]:
parts_num = 3
part_prefix = 'image_part'
split_feature='image'

In [22]:
mnist_test_splitted = mnist['test'].map(partial(split_vertically, parts=parts_num, part_prefix=part_prefix,
                                               split_feature=split_feature), remove_columns='image') #, features='image')
mnist_test_splitted

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1984/1984 [00:01<00:00, 1851.82ex/s]


Dataset({
    features: ['label', 'image_part_0', 'image_part_1', 'image_part_2'],
    num_rows: 1984
})

In [23]:
mnist_test_splitted[1]['image_part_0'] #, mnist_test_splitted[1]['image_part_1'], mnist_test_splitted[1]['image_part_2']

## Split the whole dataset:

In [24]:
def split_dataset_dict(ds_dict, parts=3, split_feature='image', part_prefix='image_part'):
    
    # we assume that datasets in the dict have the same features
    common_features = [ff for ff in list(ds_dict.values())[0].features.keys() if (ff != split_feature)]
    
    splited_dss = {}
    for val, ds in ds_dict.items():
        ds_splited = ds.map(partial(split_vertically, parts=parts_num, part_prefix=part_prefix,
                                               split_feature=split_feature), remove_columns=split_feature)
        splited_dss[val] = ds_splited
        
    splitted_features = [ff for ff in splited_dss[list(splited_dss.keys())[0]].features.keys() if ff not in common_features]
    
    # seems that only the remove feature we can use here
    parts = []
    for ff in splitted_features:
        features_to_remove = [ee for ee in splitted_features if ee!=ff]
        new_ds_dict = datasets.DatasetDict()
        
        for val, ds in splited_dss.items():
            new_ds_dict[val] = ds.remove_columns(features_to_remove)
            
        parts.append(new_ds_dict)
    return parts

In [25]:
rr = split_dataset_dict(mnist, parts=3, split_feature='image', part_prefix='image_part')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1984/1984 [00:01<00:00, 1939.64ex/s]


In [40]:
for ii in range(1, len(rr)):
    if 'label' in rr[ii]['test'].column_names:
        rr[ii] = rr[ii].remove_columns('label')

In [41]:
rr

[DatasetDict({
     train: Dataset({
         features: ['label', 'image_part_0'],
         num_rows: 11982
     })
     test: Dataset({
         features: ['label', 'image_part_0'],
         num_rows: 1984
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image_part_1'],
         num_rows: 11982
     })
     test: Dataset({
         features: ['image_part_1'],
         num_rows: 1984
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image_part_2'],
         num_rows: 11982
     })
     test: Dataset({
         features: ['image_part_2'],
         num_rows: 1984
     })
 })]

## Save the whole dataset:

### Saving parameters:

In [42]:
save_dir = '/Users/alexgrig/data/mnist_binary38_parts3'

In [43]:
def save_splitted_dataset(ds_list, path, part_dir_name='part_', clean_dir=False):
    path = Path(path)
    
    if not path.exists():
        path.mkdir()
    
    #clean the directory:
    if clean_dir:
        for pt in path.glob('*'):
            if pt.is_file():
                pt.unlink()
            elif pt.is_dir():
                shutil.rmtree(pt)

    for ii,ds in enumerate(ds_list):
        part_dir =  f'{part_dir_name}{ii}'
        
        part_path = path / part_dir
        if part_path.exists():
            raise IOError('Directory already exists')
            
        part_path.mkdir()
        
        ds.save_to_disk(part_path)
        

In [44]:
save_splitted_dataset(rr, path=save_dir, clean_dir=False)

In [45]:
def load_splitted_part(path, part_no, part_prefix = 'part_', split_feature_prefix='image', new_name_split_feature=None):
    path = Path(path)
    
    part_path = path / f'{part_prefix}{part_no}'
    
    part_ds = datasets.load_from_disk(part_path)
    
    #import pdb; pdb.set_trace()
    
    if new_name_split_feature is not None:
        for kk,ds in part_ds.items():
            rename_feature = [str(ff) for ff in list(ds.features) if split_feature_prefix in str(ff)][0]
            
            part_ds[kk] = ds.rename_column(rename_feature, new_name_split_feature)
            
    return part_ds

In [46]:
load_splitted_part(save_dir, 2, part_prefix = 'part_', split_feature_prefix='image', new_name_split_feature=None)

DatasetDict({
    train: Dataset({
        features: ['image_part_2'],
        num_rows: 11982
    })
    test: Dataset({
        features: ['image_part_2'],
        num_rows: 1984
    })
})

In [59]:
rr[0].num_rows #['train']['image_part_0']

{'train': 11982, 'test': 1984}

In [70]:
rr[2]['train']['image_part_2'][0]

In [75]:
type(rr[0]['train']['label'][0])

int

In [77]:
np.array(rr[0]['train']['label'][0]).shape

()

In [85]:
list(rr[0].keys())

['train', 'test']